# Segmenting and Clustering Neighborhoods in Toronto

## Written by: Adedamola Adedokun

### DATA SCRAPING FROM WIKIPEDIA AND TRANSFORMATION TO PANDAS DATAFRAME
#### Import required libraries

In [1]:
# import the library we use to open URLs
import urllib.request

# import the BeautifulSoup library so we can parse HTML and XML documents
from bs4 import BeautifulSoup

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                       

### First scraping method

#### Download data (Toronto Postal Codes Table) from wikipedia:
https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M

In [2]:
# specify which URL/web page we are going to be scraping
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [3]:
page = urllib.request.urlopen(url)

In [5]:
# parse the HTML from our URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(page, "lxml")

In [ ]:
#View HTML source
print(soup.prettify())

In [ ]:
# use the 'find_all' function to bring back all instances of the 'table' tag in the HTML and store in 'all_tables' variable
all_tables=soup.find_all("table")
all_tables

In [ ]:
right_table=soup.find('table', class_='wikitable sortable')
right_table

In [46]:
A=[]
B=[]
C=[]


for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))
        

In [47]:

df=pd.DataFrame(A,columns=['PostalCode'])
df['Borough']=B
df['Neighborhood']=C
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [50]:
df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
df.head(10)

### Second scraping method

In [41]:

req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

soup = BeautifulSoup(req.content,'lxml')

table = soup.find_all('table')[0]

df2 = pd.read_html(str(table))

neighborhood=pd.DataFrame(df2[0]) 

In [42]:
neighborhood.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


#### Data Cleaning

In [43]:
# Remove rows with Not assigned values
neighborhood = neighborhood_r[neighborhood_r['Borough'] != 'Not assigned'].reset_index(drop=True)

neighborhood.head(20)

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,Malvern / Rouge
7,M3B,North York,Don Mills
8,M4B,East York,Parkview Hill / Woodbine Gardens
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [44]:
# replace '/' with ',' in Neighborhood column

neighborhood = neighborhood.replace({'/': ','}, regex=True)
neighborhood.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [45]:
neighborhood.shape

(103, 3)

### Getting the latitude and longitude coordinates of each neighborhood

In [46]:
geoData = pd.read_csv('http://cocl.us/Geospatial_data')
geoData.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [47]:
# Unify the 'Postal Code' column in both dataframes 
neighborhood.rename(columns={'Postal code':'Postcode'},inplace=True)
geoData.rename(columns={'Postal Code':'Postcode'},inplace=True)

In [48]:
# merge both neighborhood dataframe and geodata dataframe with common column name 'Postcode'
neighborhood = pd.merge(neighborhood, geoData, on='Postcode')
neighborhood.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


In [49]:
neighborhood.rename(columns={'Postcode':'PostalCode'},inplace=True)
neighborhood.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


In [26]:

print('This dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhood['Borough'].unique()),
        neighborhood.shape[0]
    )
)



This dataframe has 10 boroughs and 103 neighborhoods.


#### EXPLORE AND CLUSTER NEIGHBORHOODS IN TORONTO

In [16]:

# Extract the geo data (longitude and latitude) for Toronto from geopy

address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))



The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods 

In [50]:
#create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhood['Latitude'], neighborhood['Longitude'], neighborhood['Borough'], neighborhood['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        ).add_to(map_toronto)  
    
map_toronto

In [ ]:
# list the 10 borough names in Toronto
neighborhood.Borough.unique()

#### Analyzing Downtown Toronto

In [ ]:
#create Downtown Toronto dataframe
d_toronto = neighborhood[neighborhood['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
d_toronto.head()

In [ ]:
#getting geographical data (longitude and latitude) for Downtown Toronto from geopy

address = 'Downtown Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

In [ ]:
#create map ofDowntown Toronto using latitude and longitude values
map_dtoronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(ngeod['Latitude'], ngeod['Longitude'], ngeod['Borough'], ngeod['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        ).add_to(map_dtoronto)  
    
map_dtoronto

#### Define Foursquare Credentials and Version

In [ ]:
CLIENT_ID = '3V1UGCG3TRVYGV0UCAYAHS0C1WYC5T22SGZE1SVKITR3FE5I' # your Foursquare ID
CLIENT_SECRET = 'OUZK24RTZJCY3K2GL5ZZESMVXLDWBDICE3JUWIN24MI1RMG5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
#get neighborhood's name
d_toronto.loc[0, 'Neighborhood']

In [ ]:
#get neighborhood's location
neighborhood1_latitude = d_toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood1_longitude = d_toronto.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood1_name = d_toronto.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood1_name, 
                                                               neighborhood1_latitude, 
                                                               neighborhood1_longitude))

In [ ]:
# get request
radius=500
LIMIT=100
url1 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood1_latitude, 
    neighborhood1_longitude, 
    radius, 
    LIMIT)

url1

In [ ]:
results1 = requests.get(url1).json()
results1

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
venues = results1['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

In [ ]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
d_toronto_venues = getNearbyVenues(names=d_toronto['Neighborhood'],
                                   latitudes=d_toronto['Latitude'],
                                   longitudes=d_toronto['Longitude']
                                  )

In [ ]:
print(d_toronto_venues.shape)
d_toronto_venues.head()

In [ ]:
#check many venues were returned for each neighborhood
d_toronto_venues.groupby('Neighborhood').count()

In [ ]:
#check how many many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(d_toronto_venues['Venue Category'].unique())))

In [ ]:
# one hot encoding
d_toronto_onehot = pd.get_dummies(d_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
d_toronto_onehot['Neighborhood'] = d_toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns1 = [d_toronto_onehot.columns[-1]] + list(d_toronto_onehot.columns[:-1])
d_toronto_onehot = d_toronto_onehot[fixed_columns1]

d_toronto_onehot.head(25)

In [ ]:
d_toronto_onehot.shape

In [ ]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
d_toronto_grouped = d_toronto_onehot.groupby('Neighborhood').mean().reset_index()
d_toronto_grouped

In [ ]:
d_toronto_grouped.shape

In [ ]:
#print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in d_toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = d_toronto_grouped[d_toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Create the new dataframe and display the top 10 venues for each neighborhood

In [ ]:


num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = d_toronto_grouped['Neighborhood']

for ind in np.arange(d_toronto.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(d_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()



In [ ]:
# set number of clusters
kclusters = 5

d_toronto_grouped_clustering = d_toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(d_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

In [ ]:


# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

d_toronto_merged = d_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
d_toronto_merged = d_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

d_toronto_merged.head() # check the last columns!



#### Overlay K-means clustering results on the map

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(d_toronto_merged['Latitude'], d_toronto_merged['Longitude'], d_toronto_merged['Neighborhood'], d_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters 

#### Examine Cluster 1

In [ ]:
d_toronto_merged.loc[d_toronto_merged['Cluster Labels'] == 0, d_toronto_merged.columns[[2] + list(range(5, d_toronto_merged.shape[1]))]]

#### Examine Cluster 2

In [ ]:
d_toronto_merged.loc[d_toronto_merged['Cluster Labels'] == 1, d_toronto_merged.columns[[2] + list(range(5, d_toronto_merged.shape[1]))]]

#### Examine Cluster 3

In [ ]:
d_toronto_merged.loc[d_toronto_merged['Cluster Labels'] == 2, d_toronto_merged.columns[[2] + list(range(5, d_toronto_merged.shape[1]))]]

#### Examine Cluster 4

In [ ]:
d_toronto_merged.loc[d_toronto_merged['Cluster Labels'] == 3, d_toronto_merged.columns[[2] + list(range(5, d_toronto_merged.shape[1]))]]

#### Examine Cluster 5

In [ ]:
d_toronto_merged.loc[d_toronto_merged['Cluster Labels'] == 4, d_toronto_merged.columns[[2] + list(range(5, d_toronto_merged.shape[1]))]]


### OBSERVATIONS:¶

#### Cluster 1 is related to a park which is popular for outdoor activities
#### Cluster 3 is obviously linked to the City Airport and related to transportation
#### Cluster 5 is the heart of Toronto where is popular with many cafes, coffee shops and a phletora of restaurants
